In [5]:
import gymnasium as gym
import numpy as np
from gymnasium import spaces
import torch
import torch.nn.functional as F

class BinaryHologramEnv(gym.Env):
    def __init__(self, target_function, max_steps=1000000, T_PSNR=30, T_steps=100):
        """
        target_function: 타겟 이미지와의 손실(MSE 또는 PSNR) 계산 함수.
        max_steps: 최대 타임스텝 제한.
        T_PSNR: 목표 PSNR 값.
        T_steps: PSNR 목표를 유지해야 하는 최소 타임스텝.
        """
        super(BinaryHologramEnv, self).__init__()

        # Observation space: 연속형 데이터
        self.observation_space = spaces.Box(
            low=0,
            high=1,
            shape=(1, 8, 1024, 1024),  # 8채널 데이터
            dtype=np.float32,
        )

        # Action space: MultiBinary 데이터
        self.action_space = spaces.MultiBinary(1 * 8 * 1024 * 1024)

        # 목표 함수
        self.target_function = target_function

        # 에피소드 설정
        self.max_steps = max_steps
        self.T_PSNR = T_PSNR
        self.T_steps = T_steps

        # 학습 상태
        self.state = None  # MultiBinary 형식의 환경 상태
        self.observation = None  # 에이전트가 관찰할 수 있는 정보
        self.steps = 0
        self.psnr_sustained_steps = 0  # PSNR 목표 유지 스텝 수

    def reset(self, seed=None, options=None):
        """
        환경 상태를 초기화.
        """
        # 관찰값 초기화 (연속형 데이터)
        self.observation = np.random.rand(1, 8, 1024, 1024).astype(np.float32)

        # 상태 초기화 (관찰값을 0.5 기준으로 이진화)
        self.state = (self.observation >= 0.5).astype(np.int8)

        # 초기화된 타임스텝
        self.steps = 0
        self.psnr_sustained_steps = 0

        return self.observation, {}

    # def calculate_psnr(self, mse):
        """
        MSE로부터 PSNR 계산.
        """
        # if mse == 0:
            # return np.inf
        # return 10 * np.log10(1.0 / mse)  # MAX 신호 값(1.0) 기준

    def step(self, action, lr=1e-4, z=2e-3):
        """
        action: 1024x1024x8 배열로 출력된 MultiBinary 행동 데이터 (0 또는 1).
        """
        # Action을 MultiBinary 형식으로 변환
        action = np.reshape(action, (1, 8, 1024, 1024)).astype(np.int8)

        # 타겟 계산 (예: 모델이 사용할 데이터셋)
        target = train_dataset
        # target = self.state.mean(axis=-1, keepdims=True)  # 타겟 이미지 (평균값)

        # 시뮬레이션 계산
        binary = action
        sim = tt.simulate(binary, z).abs()**2
        result = torch.mean(sim, dim=1, keepdim=True)

        # MSE 계산 (이진 출력의 평균과 타겟 비교)
        mse = tt.relativeLoss(result, target, F.mse_loss).detach().cpu().numpy()
        # mse = np.mean((action.mean(axis=-1) - target.squeeze()) ** 2)

        # PSNR 계산
        psnr = tt.relativeLoss(result, target, tm.get_PSNR)
        # psnr = self.calculate_psnr(mse)

        # 보상 설계 (MSE 기반)
        reward = -mse

        # 상태 업데이트 (MultiBinary 상태 XOR로 변경)
        self.state = np.logical_xor(self.state, action).astype(np.int8)

        # 종료 조건 초기화
        terminated = False
        truncated = False
        self.steps += 1

        # 시간 초과 조건
        if self.steps >= self.max_steps:
            truncated = True

        # 성능 기반 종료 조건 (PSNR 유지)
        if psnr >= self.T_PSNR:
            self.psnr_sustained_steps += 1
        else:
            self.psnr_sustained_steps = 0

        # 목표 PSNR을 일정 시간 유지하면 종료
        if self.psnr_sustained_steps >= self.T_steps:
            terminated = True

        # 관찰값은 상태와 독립적으로 유지됨
        observation = self.observation  # 관찰값은 행동과 무관하게 유지

        # 추가 정보 반환
        info = {
            "mse": mse,
            "psnr": psnr,
            "psnr_sustained_steps": self.psnr_sustained_steps,
        }

        return observation, reward, terminated, truncated, info


In [6]:
from stable_baselines3.common.env_checker import check_env

# 환경 인스턴스 생성
env = BinaryHologramEnv(
    target_function=None,  # target_function은 사용하지 않으므로 None
    max_steps=1000000,
    T_PSNR=30,
    T_steps=100,
)

# 환경 유효성 확인
check_env(env, warn=True)


NameError: name 'train_dataset' is not defined

In [15]:
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize
from sb3_contrib import RecurrentPPO

model = BinaryNet(num_hologram=8, in_planes=1, convReLU=False,
                  convBN=False, poolReLU=False, poolBN=False,
                  deconvReLU=False, deconvBN=False).cuda()
model.load_state_dict(torch.load('result/2024-12-07 19:38:09.105795_pre_reinforce_8_0.002/2024-12-07 19:38:09.105795_pre_reinforce_8_0.002'))
model.eval()

# Create the custom Gym environment
env = BinaryHologramEnv(model=model, validloader=validloader, max_steps=1000, T_PSNR=30, T_steps=100)

# Create a vectorized environment
venv = make_vec_env(lambda: env, n_envs=1)
venv = VecNormalize(venv, norm_obs=True, norm_reward=True, clip_obs=10.0)


# Recurrent PPO 모델 학습
ppo_model = RecurrentPPO(
    "MlpLstmPolicy",
    venv,
    verbose=1,
    n_steps=2048,
    batch_size=64,
    gamma=0.99,
    learning_rate=3e-4,
    tensorboard_log="./ppo_lstm/"
)

# 모델 학습
ppo_model.learn(total_timesteps=100000)

# 학습된 모델 저장
ppo_model.save("recurrent_ppo_binary_hologram")


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.8/dist-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


KeyboardInterrupt: 